In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DoubleType

In [0]:
name_schema=StructType(fields=[StructField("forename",StringType(),True),
                               StructField("surname",StringType(),True)])

In [0]:
driver_schema=StructType(fields=[StructField("driverId",IntegerType(),False),
                                    StructField("driverRef",StringType(),True),
                                    StructField("number",IntegerType(),True),
                                    StructField("code",StringType(),True),
                                    StructField("name",name_schema),
                                    StructField("dob",StringType(),True),
                                    StructField("nationality",StringType(),True),
                                    StructField("url",StringType(),True)])

In [0]:
driver_df=spark.read\
    .schema(driver_schema)\
        .json("/mnt/storageformula1training/raw/drivers.json")

In [0]:
display(driver_df)

In [0]:
from pyspark.sql.functions import current_timestamp, concat, col,lit
driver_renamed_df=driver_df.withColumnRenamed("driverId","driver_id").withColumnRenamed("driverRef","driver_ref").withColumn("ingestion_date",current_timestamp())\
                           .withColumn("name",concat(col("name.forename"),lit(" "),col("name.surname")))
display(driver_renamed_df)


In [0]:
driver_drop_df=driver_renamed_df.drop("url")
display(driver_drop_df)

In [0]:
driver_final_df=driver_drop_df.write.mode("overwrite").parquet("/mnt/storageformula1training/processed/drivers")

In [0]:
%fs
ls /mnt/storageformula1training/processed/drivers